# Deploy Best Model to an Online Endpoint

Use this notebook to convert the best trained model into an MLflow artifact, register it with Azure ML, and deploy it to a managed online endpoint.

## 0. Prerequisites

Make sure you have:

- Run `run_pipeline.py` (or the pipeline from `main.ipynb`) so that `outputs/model_output/<model_name>_model.pkl` exists locally.
- `azure-ai-ml>=1.14.0`, `mlflow`, and `azure-identity` installed in the current environment.
- `config.env` populated with your workspace and data asset settings.

If you are on a compute instance, these requirements should already be satisfied.


In [14]:
from __future__ import annotations

import json
import os
import time
from pathlib import Path
from typing import Dict, Any

import joblib
import mlflow
import mlflow.sklearn
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Model, ManagedOnlineEndpoint, ManagedOnlineDeployment
from azure.ai.ml.constants import AssetTypes


NOTEBOOK_ROOT = Path.cwd().resolve()
PROJECT_ROOT = NOTEBOOK_ROOT if (NOTEBOOK_ROOT / "src").exists() else NOTEBOOK_ROOT.parent
os.chdir(PROJECT_ROOT)

print(f"Project root: {PROJECT_ROOT}")


Project root: /workspaces/customer-churn-prediction-azureml


In [ ]:
# --- User Inputs -----------------------------------------------------------
MODEL_OUTPUT_DIR = PROJECT_ROOT / "outputs" / "model_output"
MODEL_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Allow overriding the exact pickle path via env var; otherwise auto-detect the newest *_model.pkl
env_model_path = os.getenv("AML_MODEL_PICKLE_PATH")
if env_model_path:
    MODEL_PICKLE_PATH = Path(env_model_path)
else:
    candidate_pkls = sorted(
        MODEL_OUTPUT_DIR.glob("*_model.pkl"),
        key=lambda p: p.stat().st_mtime,
        reverse=True,
    )
    if not candidate_pkls:
        available = "\n".join(str(p) for p in MODEL_OUTPUT_DIR.glob("*")) or "(directory is empty)"
        raise FileNotFoundError(
            "No *_model.pkl files were found in outputs/model_output. \n"
            "Run run_pipeline.py locally or download the best-model artifacts from the pipeline run, "
            "then place them inside outputs/model_output or set AML_MODEL_PICKLE_PATH to the file you want to use.\n"
            f"Current directory listing:\n{available}"
        )
    MODEL_PICKLE_PATH = candidate_pkls[0]

# Directory to store the temporary MLflow model artifact
MLFLOW_MODEL_DIR = PROJECT_ROOT / "artifacts" / "mlflow_online_model"
MLFLOW_MODEL_DIR.mkdir(parents=True, exist_ok=True)

# Names for Azure resources
MODEL_NAME = os.getenv("AML_DEPLOY_MODEL_NAME", "bank-churn-best-model")
ENDPOINT_NAME = os.getenv("AML_ONLINE_ENDPOINT_NAME", f"churn-endpoint-{int(time.time())}")
DEPLOYMENT_NAME = os.getenv("AML_ONLINE_DEPLOYMENT_NAME", "blue")

print(f"Using model pickle: {MODEL_PICKLE_PATH}")
print(f"MLflow artifact dir: {MLFLOW_MODEL_DIR}")
print(f"Model asset name: {MODEL_NAME}")
print(f"Endpoint name: {ENDPOINT_NAME}")
print(f"Deployment name: {DEPLOYMENT_NAME}")


FileNotFoundError: No *_model.pkl files were found in outputs/model_output. 
Run run_pipeline.py locally or download the best-model artifacts from the pipeline run, then place them inside outputs/model_output or set AML_MODEL_PICKLE_PATH to the file you want to use.
Current directory listing:
(directory is empty)

In [11]:
# --- Convert pickle -> MLflow model ---------------------------------------
if not MODEL_PICKLE_PATH.exists():
    raise FileNotFoundError(
        "Pickle file not found. Run run_pipeline.py (or pipeline cell in main.ipynb) "
        "to generate outputs/model_output/<model>_model.pkl."
    )

model = joblib.load(MODEL_PICKLE_PATH)
mlflow.sklearn.save_model(model, path=str(MLFLOW_MODEL_DIR), serialization_format="cloudpickle")
print(f"Saved MLflow model artifact to {MLFLOW_MODEL_DIR}")


FileNotFoundError: Pickle file not found. Run run_pipeline.py (or pipeline cell in main.ipynb) to generate outputs/model_output/<model>_model.pkl.

In [ ]:
# --- Connect to Azure ML workspace ----------------------------------------
load_dotenv(PROJECT_ROOT / "config.env")

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception:
    credential = InteractiveBrowserCredential()

ml_client = MLClient.from_config(credential=credential)
print(
    f"Connected to workspace: {ml_client.workspace_name} | "
    f"resource group: {ml_client.resource_group_name}"
)


In [ ]:
# --- Register MLflow model asset ------------------------------------------
model_asset = Model(
    name=MODEL_NAME,
    path=str(MLFLOW_MODEL_DIR),
    type=AssetTypes.MLFLOW_MODEL,
    description="Best churn model exported from training pipeline",
)
registered_model = ml_client.models.create_or_update(model_asset)
print(f"Registered model: {registered_model.name}:{registered_model.version}")


In [ ]:
# --- Create or update managed online endpoint -----------------------------
endpoint = ManagedOnlineEndpoint(
    name=ENDPOINT_NAME,
    auth_mode="key",
    description="Online endpoint serving the churn model",
)

endpoint = ml_client.begin_create_or_update(endpoint).result()
print(f"Endpoint ready: {endpoint.name}")


In [ ]:
# --- Deploy the model ------------------------------------------------------
deployment = ManagedOnlineDeployment(
    name=DEPLOYMENT_NAME,
    endpoint_name=ENDPOINT_NAME,
    model=registered_model,
    instance_type=os.getenv("AML_ONLINE_INSTANCE_TYPE", "Standard_DS3_v2"),
    instance_count=int(os.getenv("AML_ONLINE_INSTANCE_COUNT", "1")),
)

ml_client.online_deployments.begin_create_or_update(deployment).result()
print(f"Deployment '{DEPLOYMENT_NAME}' is live")


In [ ]:
# --- Route traffic to the deployment --------------------------------------
endpoint.traffic = {DEPLOYMENT_NAME: 100}
ml_client.begin_create_or_update(endpoint).result()
print(f"Endpoint traffic updated: {endpoint.traffic}")


## 6. Invoke the endpoint

Create a JSON file with inference data (for example `sample-data.json`) that matches the shape expected by your model. Use the cell below to invoke the endpoint and inspect predictions.


In [ ]:
REQUEST_FILE = PROJECT_ROOT / "sample-data.json"

if not REQUEST_FILE.exists():
    raise FileNotFoundError(
        f"{REQUEST_FILE} not found. Create a JSON payload (list of feature dicts) before invoking."
    )

response = ml_client.online_endpoints.invoke(
    endpoint_name=ENDPOINT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    request_file=str(REQUEST_FILE),
)
print("Raw response:", response)


## 7. (Optional) Delete the endpoint

Managed online endpoints accrue cost while running. Use the following cell to delete it when you're done testing.


In [ ]:
# Uncomment to delete the managed endpoint when no longer needed
# ml_client.online_endpoints.begin_delete(name=ENDPOINT_NAME)
# print(f"Deleted endpoint {ENDPOINT_NAME}")
